In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses, metrics

# 1. Carga de datos procesados
processed_dir = 'data/processed'
train = pd.read_csv(os.path.join(processed_dir, 'train.csv'))
test  = pd.read_csv(os.path.join(processed_dir, 'test.csv'))

# 2. Extrae número de usuarios y libros
n_users = int(max(train.user_id.max(), test.user_id.max())) + 1
n_items = int(max(train.book_id.max(), test.book_id.max())) + 1

# 3. Prepara inputs y targets
x_train = [train.user_id.values, train.book_id.values]
y_train = train.rating.values.astype(np.float32)

x_test = [test.user_id.values, test.book_id.values]
y_test = test.rating.values.astype(np.float32)

# 4. Construye el modelo
#   - Embedding de tamaño 32 para usuarios y para ítems
#   - Dos capas densas intermedias de 64 y 32 unidades
#   - Salida única con activación lineal
user_input = layers.Input(shape=(), name='user_id', dtype='int32')
item_input = layers.Input(shape=(), name='book_id', dtype='int32')

user_emb = layers.Embedding(input_dim=n_users, output_dim=32, name='user_emb')(user_input)
item_emb = layers.Embedding(input_dim=n_items, output_dim=32, name='item_emb')(item_input)

# Aplanar embeddings
user_vec = layers.Flatten()(user_emb)
item_vec = layers.Flatten()(item_emb)

# Concatenar y procesar
concat = layers.Concatenate()([user_vec, item_vec])
x = layers.Dense(64, activation='relu')(concat)
x = layers.Dense(32, activation='relu')(x)
output = layers.Dense(1, activation='linear')(x)

model = models.Model(inputs=[user_input, item_input], outputs=output)

# 5. Compila el modelo
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.MeanSquaredError(),
    metrics=[metrics.RootMeanSquaredError(), metrics.MeanAbsoluteError()]
)

model.summary()

# 6. Entrena
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=1024,
    epochs=5,
    validation_split=0.1,
    verbose=1
)

# 7. Evalúa en test
eval_results = model.evaluate(x_test, y_test, verbose=1)
print(f"Test RMSE: {eval_results[1]:.4f}, Test MAE: {eval_results[2]:.4f}")
